# Nasdaq Data Link
## ETL

In [9]:
#Importar librerías
import numpy as np;
import pandas as pd;

In [10]:
!pip install nasdaq-data-link



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
import nasdaqdatalink

In [12]:
nasdaqdatalink.read_key(filename="/data/.corporatenasdaqdatalinkapikey")

#### Extracción de datos históricos de Apple en NASDAQ 📈

In [13]:
AAPL = nasdaqdatalink.get('WIKI/AAPL', start_date='2010-01-01', end_date='2023-01-01',
                  collapse='daily')

In [14]:
AAPL

,Open,High,Low,Close,Volume,Ex-Dividend,Split Ratio,Adj. Open,Adj. High,Adj. Low,Adj. Close,Adj. Volume
Date,,,,,,,,,,,,
2010-01-04,213.43,214.50,212.38,214.010,17633200.0,0.0,1.0,27.428730,27.566240,27.293790,27.503268,123432400.0
2010-01-05,214.60,215.59,213.25,214.380,21496600.0,0.0,1.0,27.579091,27.706320,27.405597,27.550818,150476200.0
2010-01-06,214.38,215.23,210.75,210.970,19720000.0,0.0,1.0,27.550818,27.660055,27.084312,27.112585,138040000.0
2010-01-07,211.75,212.00,209.05,210.580,17040400.0,0.0,1.0,27.212826,27.244955,26.865839,27.062465,119282800.0
2010-01-08,210.30,212.00,209.06,211.980,15986100.0,0.0,1.0,27.026481,27.244955,26.867124,27.242385,111902700.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2018-03-21,175.04,175.09,171.26,171.270,35247358.0,0.0,1.0,175.040000,175.090000,171.260000,171.270000,35247358.0
2018-03-22,170.00,172.68,168.60,168.845,41051076.0,0.0,1.0,170.000000,172.680000,168.600000,168.845000,41051076.0
2018-03-23,168.39,169.92,164.94,164.940,40248954.0,0.0,1.0,168.390000,169.920000,164.940000,164.940000,40248954.0


In [15]:
AAPL.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2070 entries, 2010-01-04 to 2018-03-27
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Open         2070 non-null   float64
 1   High         2070 non-null   float64
 2   Low          2070 non-null   float64
 3   Close        2070 non-null   float64
 4   Volume       2070 non-null   float64
 5   Ex-Dividend  2070 non-null   float64
 6   Split Ratio  2070 non-null   float64
 7   Adj. Open    2070 non-null   float64
 8   Adj. High    2070 non-null   float64
 9   Adj. Low     2070 non-null   float64
 10  Adj. Close   2070 non-null   float64
 11  Adj. Volume  2070 non-null   float64
dtypes: float64(12)
memory usage: 210.2 KB


In [18]:
AAPL.columns

Index(['Open', 'High', 'Low', 'Close', 'Volume', 'Ex-Dividend', 'Split Ratio',
       'Adj. Open', 'Adj. High', 'Adj. Low', 'Adj. Close', 'Adj. Volume'],
      dtype='object')

#### Descarga de Tabla en formato CSV 🔽

In [19]:
AAPL.to_csv('AAPL.csv')

### Conexión de Redshift y carga de tabla

In [20]:
!pip install psycopg2


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [21]:
import psycopg2

In [22]:

url="data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws.com"
data_base="data-engineer-database"
user="gerasanchezcorzo_coderhouse"
with open("C:/Users/sanch/Downloads/Data engeniering/pwd.txt", "r") as f:
    pwd=f.read()

try:
    conn = psycopg2.connect(
        host='data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws.com',
        dbname=data_base,
        user=user,
        password=pwd,
        port='5439'
    )
    print("Connected to Redshift successfully!")
    
except Exception as e:
    print("Unable to connect to Redshift.")
    print(e)

Connected to Redshift successfully!


In [25]:
from psycopg2.extras import execute_values
cur = conn.cursor()
# Define the table name
table_name = 'aapl'
# Define the columns you want to insert data into
columns = ['Open', 'High', 'Low', 'Close', 'Volume', 'Ex-Dividend', 'Split Ratio', 'Adj. Open', 'Adj. High', 'Adj. Low', 'Adj. Close', 'Adj. Volume']
# Generate 
values = [tuple(x) for x in AAPL.to_numpy()]
insert_sql = f"INSERT INTO {table_name} ({', '.join(columns)}) VALUES %s"
# Execute the INSERT statement using execute_values
cur.execute("BEGIN")
execute_values(cur, insert_sql, values)
cur.execute("COMMIT")

InFailedSqlTransaction: current transaction is aborted, commands ignored until end of transaction block


In [ ]:
insert_sql

In [ ]:
conn.close()